In [ ]:
import numpy as np
from meshplot import plot
import igl


def loop_gps(v, f, c, n):
    # transform to 9D dual-space vertices
    qq1 = np.zeros((len(v), 3))
    qq2 = np.zeros((len(v), 3))
    qlin = np.zeros((len(v), 3))
    for i, cov in enumerate(c):
        ic = np.linalg.inv(cov)
        icf = ic.flatten()
        qq1[i] = [icf[0],icf[1],icf[2]]
        qq2[i] = [icf[4],icf[5],icf[8]]
        qlin[i] = ic @ v[i]

    # perform Gaussian-product subdivision
    # note: igl.loop only handles 3D subdivs, so we split the 9D meshes into three 3D ones
    for _ in range(n):
        qq1, dmy = igl.loop(qq1, f)
        qq2, dmy = igl.loop(qq2, f)
        qlin, f  = igl.loop(qlin, f)
    
    # transform back to 3D
    v = np.zeros((len(qlin), 3))
    for i, ql in enumerate(qlin):
        icov = [qq1[i],
                [qq1[i][1], qq2[i][0], qq2[i][1]],
                [qq1[i][2], qq2[i][1], qq2[i][2]]]
        v[i] = np.linalg.inv(icov) @ ql
        
    return v, f

In [ ]:
# create a cone
v = np.array([
    [0, 1.5, 0],
    [1.41421, -1.5, -1.41421],[0, -1.5, -2], [-1.41421, -1.5, -1.41421],[-2, -1.5, 0],
    [-1.41421, -1.5, 1.41421], [0, -1.5, 2], [1.41421, -1.5, 1.41421],[2, -1.5, 0]
])

f = np.array([[0, 1, 2],[0, 2, 3], [0, 3, 4], [0, 4, 5],[0, 5, 6],[0, 6, 7],[0, 7, 8],[0, 8, 1]])

plot(v,f);

In [ ]:
# initialize with isotropic covariances
c = [np.identity(3)*0.01 for i in range(len(v))]
vs, fs = loop_gps(v, f, c, 5)

# make apex covariance sharper
c[0] *= 0.001
vs2, fs2 = loop_gps(v, f, c, 5)

# make apex covariance vertically anisotropic (reduce variance in x direction)
c[0] = [[0.001,0,0],
        [0,0.01,0],
        [0,0,0.01]]
vs3, fs3 = loop_gps(v, f, c, 5)

# make apex covariance horizontally anisotropic (reduce variance in y direction)
c[0] = [[0.01,0,0],
        [0,0.001,0],
        [0,0,0.01]]
vs4, fs4 = loop_gps(v, f, c, 5)

# make apex covariance horizontally anisotropic and super flat
c[0] = [[1,0,0],
        [0,0.00001,0],
        [0,0,1]]
vs5, fs5 = loop_gps(v, f, c, 5)


# plot all the shapes next to each other
vo = np.array([4,0,0])
fo = len(vs)
C = np.concatenate((vs, vs2 + vo, vs3 + 2*vo, vs4 + 3*vo, vs5 + 4*vo))
F = np.concatenate((fs, fs2 + fo, fs3 + 2*fo, fs4 + 3*fo, fs5 + 4*fo))

p = plot(C, F, shading={'flat':True, 'scale':2}, return_plot=True);
p.add_lines(v[f[:,0]], v[f[:,1]], shading={"line_color": "black"});
p.add_lines(v[f[:,1]], v[f[:,2]], shading={"line_color": "black"});
